# Naive Bayes Model for Newsgroups Data

For an explanation of the Naive Bayes model, see [our course notes](https://jennselby.github.io/MachineLearningCourseNotes/#naive-bayes).

This notebook uses code from http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html.

## Instructions
0. If you haven't already, follow [the setup instructions here](https://jennselby.github.io/MachineLearningCourseNotes/#setting-up-python3) to get all necessary software installed.
0. Read through the code in the following sections:
  * [Newgroups Data](#Newgroups-Data)
  * [Model Training](#Model-Training)
  * [Prediction](#Prediction)
0. Complete at least one of the following exercises:
  * [Exercise Option #1 - Standard Difficulty](#Exercise-Option-#1---Standard-Difficulty)
  * [Exercise Option #2 - Advanced Difficulty](#Exercise-Option-#2---Advanced-Difficulty)

In [1]:
from sklearn.datasets import fetch_20newsgroups # the 20 newgroups set is included in scikit-learn
from sklearn.naive_bayes import MultinomialNB # we need this for our Naive Bayes model

# These next two are about processing the data. We'll look into this more later in the semester.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## Newgroups Data

Back in the day, [Usenet](https://en.wikipedia.org/wiki/Usenet_newsgroup) was a popular discussion system where people could discuss topics in relevant newsgroups (think Slack channel or subreddit). At some point, someone pulled together messages sent to 20 different newsgroups, to use as [a dataset for doing text processing](http://qwone.com/~jason/20Newsgroups/).

We are going to pull out messages from just a few different groups to try out a Naive Bayes model.

Examine the newsgroups dictionary, to make sure you understand the dataset.

**Note**: If you get an error about SSL certificates, you can fix this with the following:
1. In Finder, click on Applications in the list on the left panel
1. Double click to go into the Python folder (it will be called something like Python 3.7)
1. Double click on the Install Certificates command in that folder


In [2]:
# which newsgroups we want to download
newsgroup_names = ['comp.graphics', 'rec.sport.hockey', 'sci.electronics', 'sci.space']

# get the newsgroup data (organized much like the iris data)
newsgroups = fetch_20newsgroups(categories=newsgroup_names, shuffle=True, random_state=265)

newsgroups.keys()


dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

This next part does some processing of the data, because the scikit-learn Naive Bayes module is expecting numerical data rather than text data. We will talk more about what this code is doing later in the semester. For now, you can ignore it.

In [3]:
# Convert the text into numbers that represent each word (bag of words method)
word_vector = CountVectorizer()
word_vector_counts = word_vector.fit_transform(newsgroups.data)

# Account for the length of the documents:
#   get the frequency with which the word occurs instead of the raw number of times
term_freq_transformer = TfidfTransformer()
term_freq = term_freq_transformer.fit_transform(word_vector_counts)

## Model Training

Now we fit the Naive Bayes model to the subset of the 20 newsgroups data that we've pulled out.

In [4]:
# Train the Naive Bayes model
model = MultinomialNB().fit(term_freq, newsgroups.target)

## Prediction

Let's see how the model does on some (very short) documents that we made up to fit into the specific categories our model is trained on.

In [5]:
# Predict some new fake documents
fake_docs = [
    'That GPU has amazing performance with a lot of shaders mars',
    'The player had a wicked slap shot',
    'I spent all day yesterday soldering banks of capacitors',
    'Today I have to solder a bank of capacitors',
    'NASA has rovers on Mars',


#     'performance GPU',
#     'player shot ball',
#     'soldering capacitors',
#     'today have',
#     'rovers mars NASA'

    ]
fake_counts = word_vector.transform(fake_docs)

fake_term_freq = term_freq_transformer.transform(fake_counts)

predicted = model.predict(fake_term_freq)
print('Predictions:')
for doc, group in zip(fake_docs, predicted):
    print('\t{0} => {1}'.format(doc, newsgroups.target_names[group]))

probabilities = model.predict_proba(fake_term_freq)
print('Probabilities:')
print(''.join(['{:17}'.format(name) for name in newsgroups.target_names]))
for probs in probabilities:
    print(''.join(['{:<17.8}'.format(prob) for prob in probs]))

Predictions:
	That GPU has amazing performance with a lot of shaders mars => sci.space
	The player had a wicked slap shot => rec.sport.hockey
	I spent all day yesterday soldering banks of capacitors => sci.space
	Today I have to solder a bank of capacitors => sci.electronics
	NASA has rovers on Mars => sci.space
Probabilities:
comp.graphics    rec.sport.hockey sci.electronics  sci.space        
0.24161308       0.18702187       0.2129319        0.35843316       
0.12948055       0.51155698       0.18248712       0.17647535       
0.18604814       0.24117771       0.27540452       0.29736963       
0.21285086       0.21081302       0.3486507        0.22768541       
0.079185633      0.066225915      0.10236622       0.75222223       


In [6]:
# for doc, group in zip(fake_docs, predicted):
# print(newsgroups.data[1:])

# Exercise Option #1 - Standard Difficulty

Modify the fake documents and add some new documents of your own. 

What words in your documents have particularly large effects on the model probabilities? Note that we're not looking for documents that consist of a single word, but for words that, when included or excluded from a document, tend to change the model's output.



---
Unsurprisingly, words like 'mars' or 'shaders' modify the models prediction immensely. These are words that show up in one categorie, and rarely show up in the others.

# Exercise Option #2 - Advanced Difficulty

Write some code to count up how often the words you found in the exercise above appear in each category in the training dataset. Does this match up with your intuition?

In [24]:
# store variables to make code slightly less horrific
cat_comp = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=['comp.graphics'])
cat_sports = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=['rec.sport.hockey'])
cat_elec = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=['sci.electronics'])
cat_space = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=['sci.space'])

cats = [cat_comp, cat_sports, cat_elec, cat_space]

done


In [41]:
import inspect, re

for doc in fake_docs: # loop through docs
    words = doc.split(' ')
    for word in words: # loop through words in each doc
        match = re.compile(r'\b%s\b' % '\\b|\\b'.join([word]), flags=re.IGNORECASE) # make words compatible with findall
        for i,group in enumerate(cats): # loop through categories per word
            result = len(match.findall(str(group))) # find matches and store result variable
            if result != 0:
                print(newsgroup_names[i], word, result) # print result
        print("\n")


comp.graphics That 734
rec.sport.hockey That 559
sci.electronics That 528
sci.space That 783




comp.graphics has 201
rec.sport.hockey has 136
sci.electronics has 92
sci.space has 131


comp.graphics amazing 3
rec.sport.hockey amazing 5
sci.space amazing 1


comp.graphics performance 28
rec.sport.hockey performance 8
sci.electronics performance 2
sci.space performance 2


comp.graphics with 641
rec.sport.hockey with 236
sci.electronics with 251
sci.space with 319


comp.graphics a 2198
rec.sport.hockey a 986
sci.electronics a 1160
sci.space a 1451


comp.graphics lot 40
rec.sport.hockey lot 38
sci.electronics lot 22
sci.space lot 31


comp.graphics of 1915
rec.sport.hockey of 786
sci.electronics of 767
sci.space of 1745




comp.graphics mars 1
sci.space mars 65


comp.graphics The 3450
rec.sport.hockey The 2567
sci.electronics The 1982
sci.space The 3953


comp.graphics player 7
rec.sport.hockey player 73
sci.electronics player 2


comp.graphics had 35
rec.sport.hockey had 95
sci.ele